In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from keras import layers, Sequential
from keras.models import Model
from keras.layers import Input, Dense
from keras.optimizers import Adam, SGD, Adagrad, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from keras import initializers
from keras.optimizers import Adam
import os
from tensorflow.keras.preprocessing import image_dataset_from_directory
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
import matplotlib.pyplot as plt

2023-04-08 22:48:35.885769: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-08 22:48:36.231014: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-08 22:48:36.233937: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/piyush/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-04-08 22:48:

#### Reading the data, normalizing and flattening it

##### Reading and Normalizing

In [2]:

train = image_dataset_from_directory(
    'Group_24/train/',
    labels='inferred',
    label_mode='int',
    batch_size=1,
    image_size=(28, 28),
    shuffle=True,
    seed=42,
    color_mode='grayscale',
    validation_split=0.0
)

val = image_dataset_from_directory(
    'Group_24/val/',
    labels='inferred',
    label_mode='int',
    batch_size=1,
    image_size=(28, 28),
    shuffle=True,
    seed=42,
    color_mode='grayscale',
    validation_split=0.0
)

test = image_dataset_from_directory(
    'Group_24/test/',
    labels='inferred',
    label_mode='int',
    batch_size=1,
    image_size=(28, 28),
    shuffle=True,
    seed=42,
    color_mode='grayscale',
    validation_split=0.0
)

def normalize(image,label):
    image = tf.cast(image/255. ,tf.float32)
    return image, label

train = train.map(normalize)
val = val.map(normalize)
test = test.map(normalize)

Found 11385 files belonging to 5 classes.


2023-04-08 22:48:38.438750: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/piyush/.local/lib/python3.10/site-packages/cv2/../../lib64:
2023-04-08 22:48:38.438771: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-04-08 22:48:38.438787: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (piyush-HP-Spectre-x360-Convertible-13-aw2xxx): /proc/driver/nvidia/version does not exist
2023-04-08 22:48:38.438995: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other ope

Found 3795 files belonging to 5 classes.
Found 3795 files belonging to 5 classes.


##### Flattening

Preparing training tensors

In [3]:
# Iterate through the dataset and reshape each image tensor
image_tensors = []
label_tensors = []
for image, labels in train:
    num_images = image.shape[0]
    image_vectors = tf.reshape(image, [num_images, -1])
    image_tensors.append(image_vectors)
    label_tensors.append(labels)

# Concatenate the image tensors into a single tensor
train_vectors = [tf.concat(image_tensors, axis=0), tf.concat(label_tensors, axis=0)]

Preparing validation tensors

In [4]:
# Iterate through the dataset and reshape each image tensor
image_tensors = []
label_tensors = []
for image, labels in val:
    num_images = image.shape[0]
    image_vectors = tf.reshape(image, [num_images, -1])
    image_tensors.append(image_vectors)
    label_tensors.append(labels)

# Concatenate the image tensors into a single tensor
val_vectors = [tf.concat(image_tensors, axis=0), tf.concat(label_tensors, axis=0)]

Preparing testing tensors

In [5]:
# Iterate through the dataset and reshape each image tensor
image_tensors = []
label_tensors = []
for image, labels in test:
    num_images = image.shape[0]
    image_vectors = tf.reshape(image, [num_images, -1])
    image_tensors.append(image_vectors)
    label_tensors.append(labels)

# Concatenate the image tensors into a single tensor
test_vectors = [tf.concat(image_tensors, axis=0), tf.concat(label_tensors, axis=0)]

#### 1 Hidden layer autoencoder

In [7]:
initializer = initializers.RandomNormal(mean=0.0, stddev=0.05, seed=42)

k=5 # no. of classes
model_history = dict()

# train different achitectures and optimizers
print('Training 1 Hidden Layer autoencoder with different number of neurons in bottleneck layer')
for num_neurons in [32, 64, 128, 256]:
    print(f'1 Hidden Layer : {num_neurons} neurons')
    # define model
    model = Sequential([
        layers.Dense(784, activation="relu", input_shape=(784,)),
        layers.Dense(num_neurons, activation="sigmoid", name="hidden", 
                        kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
        layers.Dense(784, activation="relu", name="output", 
                        kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
    ])
    
    # compile model
    loss = tf.keras.losses.MeanSquaredError()
    adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(optimizer=adam_optimizer, loss=loss, metrics=['mse'])
    
    # callbacks
    my_callbacks = [
        EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5),
        TensorBoard(log_dir=f'./logdir/Q2/1_hidden_layer_{num_neurons}_neurons/')
    ]

    model_fit = model.fit(train_vectors[0].numpy(),train_vectors[0].numpy(), batch_size=len(train_vectors[0]), epochs=10000, verbose=0, callbacks=my_callbacks, 
                            validation_split=0.0, validation_data=(val_vectors[0].numpy(), val_vectors[0].numpy()), shuffle=True, validation_batch_size=None)
    
    model_history[f'1_hidden_layer_{num_neurons}_neurons'] = model_fit.history['mse']
    
    hist_metric = 'mse'
    print(f'epochs: {len(model_fit.history[hist_metric])}, mse: {model_fit.history[hist_metric][-1]}\n')
    model.save(f'models/Q2/1_hidden_layer_{num_neurons}_neurons.tf')



Training models with different architectures and optimizers
1 Hidden Layer : 32 neurons
epochs: 386, mse: 0.02199326641857624

1 Hidden Layer : 64 neurons
epochs: 313, mse: 0.015301888808608055

1 Hidden Layer : 128 neurons
epochs: 206, mse: 0.0163264200091362

1 Hidden Layer : 256 neurons
epochs: 199, mse: 0.015806687995791435



Displaying reconstructed image

In [ ]:
# Group_24/train/0/img_1.jpg
# Group_24/test/0/img_23.jpg
# Group_24/val/0/img_111.jpg


#### 3 Hidden layer autoencoder

In [8]:
initializer = initializers.RandomNormal(mean=0.0, stddev=0.05, seed=42)

k=5 # no. of classes
model_history = dict()

# train different achitectures and optimizers
print('Training 3 Hidden Layer autoencoder with different number of neurons in bottleneck layer')
for num_neurons in [32, 64, 128, 256]:
    print(f'1 Hidden Layer : {num_neurons} neurons')
    # define model
    model = Sequential([
        layers.Dense(784, activation="relu", input_shape=(784,)),
        layers.Dense(400, activation="sigmoid", name="hidden1", 
                        kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
        layers.Dense(num_neurons, activation="sigmoid", name="bottleneck", 
                        kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
        layers.Dense(400, activation="sigmoid", name="hidden3", 
                        kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
        layers.Dense(784, activation="relu", name="output", 
                        kernel_initializer=initializer, bias_initializer=initializers.Zeros()),
    ])
    
    # compile model
    loss = tf.keras.losses.MeanSquaredError()
    adam_optimizer = Adam(learning_rate=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-8)
    model.compile(optimizer=adam_optimizer, loss=loss, metrics=['mse'])
    
    # callbacks
    my_callbacks = [
        EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=5),
        TensorBoard(log_dir=f'./logdir/Q2/3_hidden_layer_{num_neurons}_neurons/')
    ]

    model_fit = model.fit(train_vectors[0].numpy(),train_vectors[0].numpy(), batch_size=len(train_vectors[0]), epochs=10000, verbose=0, callbacks=my_callbacks, 
                            validation_split=0.0, validation_data=(val_vectors[0].numpy(), val_vectors[0].numpy()), shuffle=True, validation_batch_size=None)
    
    model_history[f'3_hidden_layer_{num_neurons}_neurons'] = model_fit.history['mse']
    
    hist_metric = 'mse'
    print(f'epochs: {len(model_fit.history[hist_metric])}, mse: {model_fit.history[hist_metric][-1]}\n')
    model.save(f'models/Q2/3_hidden_layer_{num_neurons}_neurons.tf')



Training 3 Hidden Layer autoencoder with different number of neurons in bottleneck layer
1 Hidden Layer : 32 neurons
epochs: 283, mse: 0.05878017470240593

1 Hidden Layer : 64 neurons
epochs: 280, mse: 0.05350026115775108

1 Hidden Layer : 128 neurons
epochs: 274, mse: 0.047249697148799896

